In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
IMG_SIZE = 224

TEST_CSV = "./data/test.csv"
TEST_DIR = "./data/test"
MODEL_CHECKPOINT = "./resnet_based_models/resnet_model_epoch_40.pth"
OUTPUT_CSV = "./resnet_modelpredictions.csv"

# Dataset class for inference
class TestPhotoDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Image transformations (same as training/validation but no augmentation)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Load test data
test_dataset = TestPhotoDataset(TEST_CSV, TEST_DIR, transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define model (same as training)
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
)
model.load_state_dict(torch.load(MODEL_CHECKPOINT, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

# Run inference
results = []

with torch.no_grad():
    for inputs, filenames in tqdm(test_loader, desc="Running Inference"):
        inputs = inputs.to(DEVICE)
        outputs = model(inputs)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).int().cpu().numpy().flatten()

        for fname, pred in zip(filenames, preds):
            results.append({"filename": fname, "predicted_label": pred})

# Save predictions to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Predictions saved to {OUTPUT_CSV}")
